# Introduction

###### Aim

Once Jean's repository has been cloned, explore the code structure, understand the necessary changes to perform, implement them.


###### Install packages 

OMPython: https://github.com/OpenModelica/OMPython. Don't forget to run with python3:

    python3 -m pip install -U https://github.com/OpenModelica/OMPython/archive/master.zip
    
###### ERROR TROUBLESHOOTING - Modelica files compilation

Warning: do not run `launch_openmodelica.py` with conda activated ! The conda compiler cannot find two required packages (*llapack*, *lblas*). Run the script without activating the conda environment, such that the standard compiler is used.

# File structure



In [1]:
!tree ../

../
├── Deep_learning
│   ├── Help_test_driver.txt
│   ├── test_driver.py
│   ├── test_runmodel.py
│   └── utils_deeplearning.py
├── modelica
│   └── original
│       ├── flow_equations
│       ├── Mathcard.mo
│       ├── Mathcard.mo.BACKUP
│       ├── Mathcard_ORIGINAL.mo
│       └── parameters
│           ├── ModelParametersMHF8000
│           ├── ModelParametersMHF8000 -> ModelParametersSHF8000.diff
│           ├── ModelParametersNH
│           ├── ModelParametersNH -> ModelParametersMHF8000.diff
│           ├── ModelParametersSHF
│           ├── ModelParametersSHF8000
│           ├── ModelParametersSHF8000 -> ModelParametersSHF9000.diff
│           ├── ModelParametersSHF9000
│           ├── ModelParametersSHF -> ModelParametersSHF8000.diff
│           ├── Ursino1998Model
│           ├── Ursino1998Model -> Ursino1998ModelVad.diff
│           ├── Ursino1998ModelVad
│           ├── Ursino1998Model_VAD2
│           └── Ursino1998ModelVad -> Ursino1998Model_VAD2.diff
├── notebooks
│   ├

###### Comments

* Code to the original modelica file `Modelica_Code/0D_Original/Matcard.mo`
* The first step is to run simulations of Modelica code with `launch_openmodelica.py` (I guess this is the new version of  `launch_openmodelica_physiological.py`)
* The run pre-processing
* Eventually, DNN and post-processing (or post-processing and DNN...)

# Run simulations

File: `Simulation_script/launch_openmodelica.py`

###### Data management

The `filepath` variable should be set. Its current value is:

    filepath="/Users/jean.bonnemain/Documents/Code/0d_model/Modelica_Code/0D_Original/"
    
The modelica file is assumed to be named `Mathcard.mo` inside this folder. For each sample, some parameters are generated and a new modelica file is created.l

###### Steps performed in code

1. Define `filepath` 
1. Determine the output folder, based on `os.getcwd()` and curent time !
    1. Call `utils_openmodelica.prepareOutputFolder(outputFolder)`, which creates the output folder containing the subfolders `models/` and `outputs/`
1. Define parameters and their range of values
1. Loop over `numberofsamples`
    1. `uo.launchSimulation(filepath, listParameters, suffix, outputfolder)`
    1. Output file format: `Mathcard_suffix.mo`
    1. `changeValueInFile(filepath, listParameters, outputfile, outputfolder)`
    1. Load modelica libraries with `OMPython.OMCSessionZMQ.sendExpression`
    1. Use the same function to simulate the file whose parameters were modified
    1. The resulting file of the simulation `<file>_res.mat` is renamed and placed in an `output` folder

## What to change 

### File path

File `launch_openmodelica.py`, variable `filepath`.

## `changeValueInFile`

Differences between the original Mathcard file and `Mathcard_0.mo`:

    --- /media/matthias/Data/Programmation/git/VAD-0D-DNN/modelica/original/Mathcard.mo	Fri Mar 13 11:35:24 2020
    +++ /media/matthias/Data/Programmation/git/VAD-0D-DNN/modelica/original/Mathcard_0.mo	Fri Mar 13 12:20:44 2020
    @@ -844,13 +844,13 @@
             parameter Real Param_MitralicValve_R(unit = "mmHg.s2/ml") = 2.5 * 0.001;
             parameter Real Param_LeftVentricle_V0(unit = "ml") = 16.77;
             parameter Real Param_LeftVentricle_kR(unit = "s/ml") = 3.75 * 0.0001;
    -        parameter Real Param_LeftVentricle_Emax0 = 2.95;
    -        parameter Real Param_LeftVentricle_EmaxRef0 = 2.392;
    -        parameter Real Param_LeftVentricle_AGain_Emax = 0.475;
    +        parameter Real Param_LeftVentricle_Emax0 = 2.53434011308;
    +        parameter Real Param_LeftVentricle_EmaxRef0 = 2.06805532326;
    +        parameter Real Param_LeftVentricle_AGain_Emax = 0.264229438397;
             parameter Real Param_LeftVentricle_ADelay_Emax = 2;
             parameter Real Param_LeftVentricle_ATau_Emax = 8;
             parameter Real Param_LeftVentricle_P0(unit = "mmHg") = 1.5;
    -        parameter Real Param_LeftVentricle_kE(unit = "1/ml") = 0.014;
    +        parameter Real Param_LeftVentricle_kE(unit = "1/ml") = 0.0112186943547;
             parameter Real Param_LeftVentricle_TSys0(unit = "s") = 0.5;
             parameter Real Param_LeftVentricle_kSys(unit = "s2") = 0.075;
             parameter Real Param_LeftVentricle_TRef0 = 0.58;


As expected, only the relevant parameters (4 in total) are modified.

# Simulate with LVAD

The provided code relies on the fact that `ModelParametersNH`is instanciated as the class defining all parameters. This is because `changeValueInFile` changes the parameters from that class. 

* We can simply make `Ursino1998Model_VAD2` instanciate `ModelParametersNH`.
* Change RPM to 4000 in `ModelParametersNH`